<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_multiple_stock_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop



<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- ASSUMES USING DOCKER, see readme for instructions

<a id='1.1'></a>
## 2.1. Add FinRL to your path. You can of course install it as a pipy package, but this is for development purposes.


In [54]:
import sys

sys.path.append("..")

In [55]:
import pandas as pd
print(pd.__version__)

1.1.5



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [56]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_plot, backtest_stats

from pprint import pprint




<a id='1.4'></a>
## 2.4. Create Folders

In [5]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [6]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [7]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

In [8]:
print(config.DOW_30_TICKER)

['AAPL', 'MSFT', 'JPM', 'V', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [9]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-01-01',
                     ticker_list = config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [10]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [11]:
processed['log_volume'] = np.log(processed.volume*processed.close)
processed['change'] = (processed.close-processed.open)/processed.close
processed['daily_variance'] = (processed.high-processed.low)/processed.close
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,log_volume,change,daily_variance
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4,0.000000,3.017278,2.684025,100.000000,66.666667,100.000000,2.791740,2.791740,0.0,21.456922,-0.098649,0.075222
1,2009-01-02,18.570000,19.520000,18.400000,15.745411,10955700,AXP,4,0.002643,3.017278,2.684025,100.000000,66.666667,100.000000,2.850651,2.850651,0.0,18.965919,-0.179391,0.071132
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4,0.001880,2.972787,2.735796,70.355711,46.771878,100.000000,2.854292,2.854292,0.0,19.287503,-0.261008,0.081907
3,2009-01-02,44.910000,46.980000,44.709999,32.978760,7117200,CAT,4,-0.000746,2.951725,2.729582,50.429389,-29.777993,43.607834,2.840654,2.840654,0.0,19.273889,-0.361786,0.068832
4,2009-01-02,16.410000,17.000000,16.250000,12.683227,40980600,CSCO,4,-0.000088,2.939568,2.746169,60.227126,-9.019317,48.357918,2.842869,2.842869,0.0,20.068890,-0.293835,0.059133


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2018-12-31
## Trade data split: 2019-01-01 to 2020-09-30

In [12]:
train = data_split(processed, '2009-01-01','2019-01-01')
trade = data_split(processed, '2019-01-01','2021-01-01')
print(len(train))
print(len(trade))

75480
15150


In [13]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
from copy import deepcopy

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pickle
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
import random

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pickle
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common import logger
import time


class StockTradingEnvV2(gym.Env):

    """
    A stock trading environment for OpenAI gym
    Parameters:
    state space: {start_cash, <owned_shares>, for s in stocks{<stock.values>}, }
        df (pandas.DataFrame): Dataframe containing data
        transaction_cost (float): cost for buying or selling shares
        hmax (int): max number of share purchases allowed per asset
        turbulence_threshold (float): Maximum turbulence allowed in market for purchases to occur. If exceeded, positions are liquidated
        print_verbosity(int): When iterating (step), how often to print stats about state of env
        reward_scaling (float): Scaling value to multiply reward by at each step.
        initial_amount: (int, float): Amount of cash initially available
        daily_information_columns (list(str)): Columns to use when building state space from the dataframe.
        out_of_cash_penalty (int, float): Penalty to apply if the algorithm runs out of cash

    action space: <share_dollar_purchases>

    TODO:
        make cash penalty proportional to total assets


    tests:
        after reset, static strategy should result in same metrics

        buy zero should result in no costs, no assets purchased
        given no change in prices, no change in asset values
    """

    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        df,
        transaction_cost_pct=3e-3,
        date_col_name="date",
        hmax=10,
        turbulence_threshold=None,
        print_verbosity=10,
        reward_scaling=1e-4,
        initial_amount=1e6,
        daily_information_cols=["open", "close", "high", "low", "volume"],
        out_of_cash_penalty=None,
        cache_indicator_data=True,
        daily_reward=None,
        cash_penalty_proportion=0.1,
        random_start=True,
    ):
        self.df = df
        self.stock_col = "tic"
        self.assets = df[self.stock_col].unique()
        self.dates = df[date_col_name].sort_values().unique()
        self.random_start = random_start

        self.df = self.df.set_index(date_col_name)
        self.hmax = hmax
        self.initial_amount = initial_amount
        if out_of_cash_penalty is None:
            out_of_cash_penalty = -initial_amount * 0.5
        self.out_of_cash_penalty = out_of_cash_penalty
        self.print_verbosity = print_verbosity
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.daily_information_cols = daily_information_cols
        self.state_space = (
            1 + len(self.assets) + len(self.assets) * len(self.daily_information_cols)
        )
        self.action_space = spaces.Box(low=-1, high=1, shape=(len(self.assets),))
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.state_space,)
        )
        self.episode = -1  # initialize so we can call reset
        self.episode_history = []
        self.printed_header = False
        self.daily_reward = daily_reward
        self.cache_indicator_data = cache_indicator_data
        self.cached_data = None
        self.cash_penalty_proportion = cash_penalty_proportion
        if self.cache_indicator_data:
            print("caching data")
            self.cached_data = [
                self.get_date_vector(i) for i, _ in enumerate(self.dates)
            ]
            print("data cached!")

    def seed(self, seed=None):
        if seed is None:
            seed = int(round(time.time() * 1000))
        random.seed(seed)

    @property
    def current_step(self):
        return self.date_index - self.starting_point

    def reset(self):
        self.seed()
        self.sum_trades = 0
        if self.random_start:
            starting_point = random.choice(range(int(len(self.dates) * 0.5)))
            self.starting_point = starting_point
        else:
            self.starting_point = 0
        self.date_index = self.starting_point
        self.episode += 1
        self.actions_memory = []
        self.transaction_memory = []
        self.state_memory = []
        self.account_information = {
            "cash": [],
            "asset_value": [],
            "total_assets": [],
            "reward": [],
        }
        init_state = np.array(
            [self.initial_amount]
            + [0] * len(self.assets)
            + self.get_date_vector(self.date_index)
        )
        self.state_memory.append(init_state)
        return init_state

    def get_date_vector(self, date, cols=None):
        if (cols is None) and (self.cached_data is not None):
            return self.cached_data[date]
        else:
            date = self.dates[date]
            if cols is None:
                cols = self.daily_information_cols
            trunc_df = self.df.loc[date]
            v = []
            for a in self.assets:
                subset = trunc_df[trunc_df[self.stock_col] == a]
                v += subset.loc[date, cols].tolist()
            assert len(v) == len(self.assets) * len(cols)
            return v

    def return_terminal(self, reason="Last Date", reward=0):

        state = self.state_memory[-1]
        self.log_step(reason=reason, terminal_reward=reward)
        reward = reward * self.reward_scaling
        # Add outputs to logger interface
        reward_pct = self.account_information["total_assets"][-1] / self.initial_amount
        logger.record("environment/total_reward_pct", (reward_pct - 1) * 100)
        logger.record(
            "environment/daily_trades",
            self.sum_trades / (self.current_step) / len(self.assets),
        )
        logger.record("environment/completed_steps", self.current_step)
        logger.record(
            "environment/sum_rewards", np.sum(self.account_information["reward"])
        )
        logger.record(
            "environment/cash_proportion",
            self.account_information["cash"][-1]
            / self.account_information["total_assets"][-1],
        )
        return state, reward, True, {}

    def log_step(self, reason, terminal_reward=None):
        if terminal_reward is None:
            terminal_reward = self.account_information["reward"][-1]
        cash_pct = (
            self.account_information["cash"][-1]
            / self.account_information["total_assets"][-1]
        )
        rec = [
            self.episode,
            self.date_index - self.starting_point,
            reason,
            f"${int(self.account_information['total_assets'][-1])}",
            f"{terminal_reward*100:0.5f}%",
            f"{cash_pct*100:0.2f}%",
        ]

        self.episode_history.append(rec)
        print(self.template.format(*rec))

    def log_header(self):
        self.template = "{0:4}|{1:4}|{2:15}|{3:10}|{4:10}|{5:10}"  # column widths: 8, 10, 15, 7, 10
        print(
            self.template.format(
                "EPISODE",
                "STEPS",
                "TERMINAL_REASON",
                "TOT_ASSETS",
                "TERMINAL_REWARD_unsc",
                "CASH_PCT",
            )
        )
        self.printed_header = True
        
    def get_reward(self):
        if self.current_step==0:
            return 0
        else:
            assets = self.account_information['total_assets'][-1]
            cash = self.account_information['cash'][-1]
            cash_penalty = max(0, (assets*self.cash_penalty_proportion-cash))
            assets -= cash_penalty
            reward = (assets/self.initial_amount)-1
            reward/=self.current_step
            return reward

    def step(self, actions):
        # let's just log what we're doing in terms of max actions at each step.
        self.sum_trades += np.sum(np.abs(actions))

        # print header only first time
        if self.printed_header is False:
            self.log_header()

        # print if it's time.
        if (self.current_step + 1) % self.print_verbosity == 0:
            self.log_step(reason="update")

        # if we're at the end
        if self.date_index == len(self.dates) - 1:
            # if we hit the end, set reward to total gains (or losses)
            return self.return_terminal(reward=self.get_reward())
        else:
            # compute value of cash + assets
            begin_cash = self.state_memory[-1][0]
            holdings = self.state_memory[-1][1 : len(self.assets) + 1]
            assert min(holdings) >= 0
            closings = np.array(self.get_date_vector(self.date_index, cols=["close"]))
            asset_value = np.dot(holdings, closings)

            # reward is (cash + assets) - (cash_last_step + assets_last_step)
            reward = self.get_reward()

            # log the values of cash, assets, and total assets
            self.account_information["cash"].append(begin_cash)
            self.account_information["asset_value"].append(asset_value)
            self.account_information["total_assets"].append(begin_cash + asset_value)
            self.account_information["reward"].append(reward)

            # multiply action values by our scalar multiplier and save
            actions = actions * self.hmax
            self.actions_memory.append(actions)

            # scale cash purchases to asset # changes
            actions = actions / closings
            self.transaction_memory.append(actions)

            # clip actions so we can't sell more assets than we hold
            actions = np.maximum(actions, -np.array(holdings))

            # compute our proceeds from sales, and add to cash
            sells = -np.clip(actions, -np.inf, 0)
            proceeds = np.dot(sells, closings)
            costs = proceeds * self.transaction_cost_pct
            coh = begin_cash + proceeds

            # compute the cost of our buys
            buys = np.clip(actions, 0, np.inf)
            spend = np.dot(buys, closings)
            costs += spend * self.transaction_cost_pct

            # if we run out of cash, end the cycle and penalize
            if (spend + costs) > coh:
                return self.return_terminal(
                    reason="CASH SHORTAGE",
                    reward=self.get_reward()
                )

            # verify we didn't do anything impossible here
            assert (spend + costs) <= coh

            # update our holdings
            coh = coh - spend - costs
            holdings_updated = holdings + actions
            self.date_index += 1
            state = (
                [coh] + list(holdings_updated) + self.get_date_vector(self.date_index)
            )
            self.state_memory.append(state)
            reward = reward * self.reward_scaling
            return state, reward, False, {}

    def get_sb_env(self):
        def get_self():
            return deepcopy(self)

        e = DummyVecEnv([get_self])
        obs = e.reset()
        return e, obs

    def get_multiproc_env(self, n=10):
        def get_self():
            return deepcopy(self)

        e = SubprocVecEnv([get_self for _ in range(n)], start_method="fork")
        obs = e.reset()
        return e, obs

    def save_asset_memory(self):
        if self.current_step == 0:
            return None
        else:
            self.account_information["date"] = self.dates[
                -len(self.account_information["cash"]) :
            ]
            return pd.DataFrame(self.account_information)

    def save_action_memory(self):
        if self.current_step == 0:
            return None
        else:
            return pd.DataFrame(
                {
                    "date": self.dates[-len(self.account_information["cash"]) :],
                    "actions": self.actions_memory,
                    "transactions": self.transaction_memory,
                }
            )


In [14]:
print(StockTradingEnvV2.__doc__)


    A stock trading environment for OpenAI gym
    Parameters:
    state space: {start_cash, <owned_shares>, for s in stocks{<stock.values>}, }
        df (pandas.DataFrame): Dataframe containing data
        transaction_cost (float): cost for buying or selling shares
        hmax (int): max number of share purchases allowed per asset
        turbulence_threshold (float): Maximum turbulence allowed in market for purchases to occur. If exceeded, positions are liquidated
        print_verbosity(int): When iterating (step), how often to print stats about state of env
        reward_scaling (float): Scaling value to multiply reward by at each step.
        initial_amount: (int, float): Amount of cash initially available
        daily_information_columns (list(str)): Columns to use when building state space from the dataframe.
        out_of_cash_penalty (int, float): Penalty to apply if the algorithm runs out of cash

    action space: <share_dollar_purchases>

    TODO:
        make cash

#### state space
The state space of the observation is as follows 

`start_cash, <owned_shares_of_n_assets>, <<indicator_i_for_asset_j> for j in assets>`

indicators are any daily measurement you can achieve. Common ones are 'volume', 'open' 'close' 'high', 'low'.
However, you can add these as needed, 
The feature engineer adds indicators, and you can add your own as well. 


In [15]:
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,log_volume,change,daily_variance
0,2009-01-02,3.067143,3.251429,3.041429,2.791740,746015200,AAPL,4,0.000000,3.017278,2.684025,100.000000,66.666667,100.000000,2.791740,2.791740,0.0,21.456922,-0.098649,0.075222
1,2009-01-02,18.570000,19.520000,18.400000,15.745411,10955700,AXP,4,0.002643,3.017278,2.684025,100.000000,66.666667,100.000000,2.850651,2.850651,0.0,18.965919,-0.179391,0.071132
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4,0.001880,2.972787,2.735796,70.355711,46.771878,100.000000,2.854292,2.854292,0.0,19.287503,-0.261008,0.081907
3,2009-01-02,44.910000,46.980000,44.709999,32.978760,7117200,CAT,4,-0.000746,2.951725,2.729582,50.429389,-29.777993,43.607834,2.840654,2.840654,0.0,19.273889,-0.361786,0.068832
4,2009-01-02,16.410000,17.000000,16.250000,12.683227,40980600,CSCO,4,-0.000088,2.939568,2.746169,60.227126,-9.019317,48.357918,2.842869,2.842869,0.0,20.068890,-0.293835,0.059133


In [16]:
information_cols = ['daily_variance', 'change', 'log_volume', 'close','day', 
                    'macd', 'rsi_30', 'cci_30', 'dx_30', 'turbulence']

e_train_gym = StockTradingEnvV2(df = train,initial_amount = 1e6,hmax = 5000, 
                                out_of_cash_penalty = 0, 
                                cache_indicator_data=True,
                                cash_penalty_proportion=0.2, 
                                reward_scaling=1,
                                daily_information_cols = information_cols, 
                                print_verbosity = 500, random_start = True)


trade = data_split(processed, '2019-01-01','2021-01-01')


e_trade_gym = StockTradingEnvV2(df = trade,initial_amount = 1e6,hmax = 5000, 
                                out_of_cash_penalty = 0, 
                                cash_penalty_proportion=0.2,
                                reward_scaling = 1, 
                                cache_indicator_data=True,
                                daily_information_cols = information_cols, 
                                print_verbosity = 500, random_start = False)



caching data
data cached!
caching data
data cached!


## Environment for Training
There are two available environments. The multiprocessing and the single processing env. 
Some models won't work with multiprocessing. 

```python
# single processing
env_train, _ = e_train_gym.get_sb_env()


#multiprocessing
env_train, _ = e_train_gym.get_multiproc_env(n = <n_cores>)
```


In [17]:
# for this example, let's do multiprocessing with n_cores-2

import multiprocessing

n_cores = multiprocessing.cpu_count() - 2

print(f"using {n_cores} cores")

#this is our training env. It allows multiprocessing
env_train, _ = e_train_gym.get_multiproc_env(n = n_cores)
# env_train, _ = e_train_gym.get_sb_env()


#this is our observation environment. It allows full diagnostics
env_trade, _ = e_trade_gym.get_sb_env()


using 6 cores
EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  



EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  

   1|  95|CASH SHORTAGE  |$934219   |-0.25112% |1.51%     
   1| 100|CASH SHORTAGE  |$930857   |-0.23751% |1.91%     
   1| 126|CASH SHORTAGE  |$959619   |-0.17860% |0.76%     
   1| 151|CASH SHORTAGE  |$1007965  |-0.10276% |3.82%     
   1| 160|CASH SHORTAGE  |$1001214  |-0.11078% |2.17%     
   1| 178|CASH SHORTAGE  |$1013950  |-0.08832% |3.12%     
   2| 116|CASH SHORTAGE  |$1028771  |-0.14579% |0.76%     
   2| 147|CASH SHORTAGE  |$1022027  |-0.12184% |0.32%     
   2| 126|CASH SHORTAGE  |$1045104  |-0.12040% |1.17%     
   2| 124|CASH SH

  23|  79|CASH SHORTAGE  |$958135   |-0.28758% |0.66%     
  22|  66|CASH SHORTAGE  |$1081616  |-0.19334% |0.66%     
  22| 111|CASH SHORTAGE  |$1043635  |-0.14029% |0.90%     
  24|  88|CASH SHORTAGE  |$1010188  |-0.20240% |1.36%     
  25| 131|CASH SHORTAGE  |$1090874  |-0.07603% |2.54%     
  22| 141|CASH SHORTAGE  |$958570   |-0.16420% |0.17%     
  23| 148|CASH SHORTAGE  |$1177722  |-0.03005% |1.13%     
  23| 110|CASH SHORTAGE  |$1001243  |-0.15931% |2.37%     
  23| 101|CASH SHORTAGE  |$1011900  |-0.18227% |0.63%     
  26|  85|CASH SHORTAGE  |$986875   |-0.21231% |3.04%     
  25| 108|CASH SHORTAGE  |$945645   |-0.21727% |0.93%     
  24|  90|CASH SHORTAGE  |$1088247  |-0.09438% |4.09%     
  24|  84|CASH SHORTAGE  |$1032506  |-0.18823% |1.54%     
  24|  96|CASH SHORTAGE  |$1006059  |-0.18709% |1.55%     
  23| 123|CASH SHORTAGE  |$944561   |-0.18452% |1.84%     
  27|  92|CASH SHORTAGE  |$1089069  |-0.11754% |1.89%     
  25|  51|CASH SHORTAGE  |$1035318  |-0.27878% |2.86%   

  44| 137|CASH SHORTAGE  |$881321   |-0.20277% |1.95%     
  48| 134|CASH SHORTAGE  |$991581   |-0.12952% |3.35%     
  45| 117|CASH SHORTAGE  |$1036759  |-0.13189% |1.57%     
  48| 100|CASH SHORTAGE  |$986296   |-0.20715% |0.39%     
  45| 126|CASH SHORTAGE  |$1036235  |-0.11906% |2.03%     
  45|  98|CASH SHORTAGE  |$1033918  |-0.15974% |1.58%     
  45| 106|CASH SHORTAGE  |$955037   |-0.21012% |1.39%     
  46| 103|CASH SHORTAGE  |$1060182  |-0.14349% |0.38%     
  49| 107|CASH SHORTAGE  |$1155264  |-0.05945% |1.05%     
  46|  94|CASH SHORTAGE  |$1177084  |-0.01433% |3.81%     
  49| 142|CASH SHORTAGE  |$1038564  |-0.10050% |2.55%     
  46| 124|CASH SHORTAGE  |$1161884  |-0.04904% |0.83%     
  47|  95|CASH SHORTAGE  |$987424   |-0.21834% |0.27%     
  50|  84|CASH SHORTAGE  |$967493   |-0.23900% |2.61%     
  50| 117|CASH SHORTAGE  |$1113939  |-0.08602% |0.74%     
  46| 151|CASH SHORTAGE  |$1082457  |-0.06891% |2.77%     
  47| 106|CASH SHORTAGE  |$1090487  |-0.10880% |1.13%   

  70|  74|CASH SHORTAGE  |$986827   |-0.24396% |3.04%     
  68|  93|CASH SHORTAGE  |$988377   |-0.22083% |0.40%     
  67|  91|CASH SHORTAGE  |$983538   |-0.21503% |1.78%     
  66| 119|CASH SHORTAGE  |$1034285  |-0.14237% |0.30%     
  70| 112|CASH SHORTAGE  |$1094731  |-0.10609% |0.49%     
  72| 102|CASH SHORTAGE  |$1022911  |-0.17170% |0.64%     
  71| 114|CASH SHORTAGE  |$964519   |-0.19964% |0.08%     
  68|  92|CASH SHORTAGE  |$975433   |-0.21340% |2.39%     
  69| 106|CASH SHORTAGE  |$969027   |-0.19385% |1.99%     
  67| 102|CASH SHORTAGE  |$936392   |-0.23683% |1.00%     
  72|  73|CASH SHORTAGE  |$1026188  |-0.23071% |1.04%     
  71| 144|CASH SHORTAGE  |$1022725  |-0.12098% |0.74%     
  73| 128|CASH SHORTAGE  |$1045168  |-0.11050% |2.15%     
  68|  87|CASH SHORTAGE  |$1046364  |-0.18256% |0.39%     
  69| 137|CASH SHORTAGE  |$1109251  |-0.07445% |0.96%     
  70| 133|CASH SHORTAGE  |$946955   |-0.17348% |1.24%     
  73|  97|CASH SHORTAGE  |$954434   |-0.20786% |3.65%   

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [18]:
agent = DRLAgent(env = env_train)

### Model PPO


In [19]:
# from torch.nn import Softsign, ReLU
ppo_params ={'n_steps': 256, 
             'ent_coef': 0.01, 
             'learning_rate': 0.00009, 
             'batch_size': 1024, 
            'gamma': 0.99}

policy_kwargs = {
#     "activation_fn": ReLU,
    "net_arch": [1024, 1024,1024, 1024,  1024], 
#     "squash_output": True
}

model = agent.get_model("ppo",  
                        model_kwargs = ppo_params, 
                        policy_kwargs = policy_kwargs, verbose = 1)

# model = model.load("scaling_reward.model", env = env_train)

{'n_steps': 256, 'ent_coef': 0.01, 'learning_rate': 9e-05, 'batch_size': 1024, 'gamma': 0.99}
Using cuda device


In [20]:
model.learn(total_timesteps = 50000, # NOTE: be careful how many timesteps you say (Default for code testing ~ 5000)
            eval_env = env_trade, 
            eval_freq = 250,
            log_interval = 1, 
            tb_log_name = '1_18_lastrun',
            n_eval_episodes = 1)

Logging to tensorboard_log/ppo/1_18_lastrun_5


/home/campus/noah.marshall2@MAIL.MCGILL.CA/anaconda3/lib/python3.6/site-packages/stable_baselines3/common/callbacks.py:321: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x7fac03dd1588> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fac0c053668>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  
   1| 499|update         |$1035115  |0.00737%  |74.54%    
   1| 504|Last Date      |$1037612  |0.00746%  |74.14%    
Eval num_timesteps=1500, episode_reward=0.00 +/- 0.00
Episode length: 505.00 +/- 0.00
New best mean reward!
----------------------------------
| environment/        |          |
|    cash_proportion  | 0.741    |
|    completed_steps  | 504      |
|    daily_trades     | 0.00488  |
|    sum_rewards      | 0.00231  |
|    total_reward_pct | 3.76     |
| eval/               |          |
|    mean_ep_length   | 505      |
|    mean_reward      | 0.00239  |
| time/               |          |
|    fps              | 71       |
|    iterations       | 1        |
|    time_elapsed     | 21       |
|    total_timesteps  | 1536     |
----------------------------------
   3| 249|CASH SHORTAGE  |$1108755  |-0.04532% |0.01%     
Eval num_timesteps=3000, episode_reward=0.02 +/- 0.00
Episode length: 250.00 +/- 0

  15| 133|CASH SHORTAGE  |$1082723  |-0.09771% |0.36%     
Eval num_timesteps=12000, episode_reward=0.02 +/- 0.00
Episode length: 134.00 +/- 0.00
-----------------------------------------
| environment/            |             |
|    cash_proportion      | 0.00357     |
|    completed_steps      | 133         |
|    daily_trades         | 0.062       |
|    sum_rewards          | 0.0157      |
|    total_reward_pct     | 8.27        |
| eval/                   |             |
|    mean_ep_length       | 134         |
|    mean_reward          | 0.0157      |
| time/                   |             |
|    fps                  | 101         |
|    iterations           | 8           |
|    time_elapsed         | 120         |
|    total_timesteps      | 12288       |
| train/                  |             |
|    approx_kl            | 0.018996075 |
|    clip_fraction        | 0.267       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.6       |
|    explained

  29| 163|CASH SHORTAGE  |$1044765  |-0.09746% |0.51%     
Eval num_timesteps=22500, episode_reward=0.03 +/- 0.00
Episode length: 164.00 +/- 0.00
New best mean reward!
-----------------------------------------
| environment/            |             |
|    cash_proportion      | 0.0051      |
|    completed_steps      | 163         |
|    daily_trades         | 0.0743      |
|    sum_rewards          | 0.0253      |
|    total_reward_pct     | 4.48        |
| eval/                   |             |
|    mean_ep_length       | 164         |
|    mean_reward          | 0.0253      |
| time/                   |             |
|    fps                  | 106         |
|    iterations           | 15          |
|    time_elapsed         | 215         |
|    total_timesteps      | 23040       |
| train/                  |             |
|    approx_kl            | 0.019115403 |
|    clip_fraction        | 0.243       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.7 

  43| 231|CASH SHORTAGE  |$1178337  |-0.02211% |0.53%     
Eval num_timesteps=33000, episode_reward=0.05 +/- 0.00
Episode length: 232.00 +/- 0.00
-----------------------------------------
| environment/            |             |
|    cash_proportion      | 0.00531     |
|    completed_steps      | 231         |
|    daily_trades         | 0.0928      |
|    sum_rewards          | 0.0468      |
|    total_reward_pct     | 17.8        |
| eval/                   |             |
|    mean_ep_length       | 232         |
|    mean_reward          | 0.0467      |
| time/                   |             |
|    fps                  | 106         |
|    iterations           | 22          |
|    time_elapsed         | 318         |
|    total_timesteps      | 33792       |
| train/                  |             |
|    approx_kl            | 0.012994904 |
|    clip_fraction        | 0.178       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.8       |
|    explained

  57| 218|CASH SHORTAGE  |$1200794  |-0.01755% |0.09%     
Eval num_timesteps=43500, episode_reward=0.06 +/- 0.00
Episode length: 219.00 +/- 0.00
-----------------------------------------
| environment/            |             |
|    cash_proportion      | 0.000927    |
|    completed_steps      | 218         |
|    daily_trades         | 0.118       |
|    sum_rewards          | 0.0646      |
|    total_reward_pct     | 20.1        |
| eval/                   |             |
|    mean_ep_length       | 219         |
|    mean_reward          | 0.0645      |
| time/                   |             |
|    fps                  | 104         |
|    iterations           | 29          |
|    time_elapsed         | 427         |
|    total_timesteps      | 44544       |
| train/                  |             |
|    approx_kl            | 0.013167566 |
|    clip_fraction        | 0.193       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.9       |
|    explained

In [21]:
model.save("ppo_time50000.model")

In [22]:
data_turbulence = processed[(processed.date<'2019-01-01') & (processed.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [23]:
insample_turbulence.turbulence.describe()

count    2516.000000
mean       33.278197
std        33.999888
min         0.000000
25%        15.233650
50%        25.166725
75%        39.289944
max       332.850050
Name: turbulence, dtype: float64

In [24]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,1)

In [25]:
turbulence_threshold

332.85005013933926

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [26]:
trade.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,log_volume,change,daily_variance
0,2019-01-02,38.722500,39.712502,38.557499,38.505024,148158800,AAPL,2,2.335191,334.139821,308.258949,47.826590,-27.772146,14.525416,314.202890,312.704339,51.409086,22.464584,-0.005648,0.029996
0,2019-01-02,93.910004,96.269997,93.769997,92.319603,4175400,AXP,2,0.775741,335.611436,305.851699,46.156688,-67.860390,22.313462,314.159711,312.579659,51.409086,19.769977,-0.017227,0.027080
0,2019-01-02,316.190002,323.950012,313.709991,314.645142,3292200,BA,2,-0.338838,336.485023,304.296566,46.806048,-72.067253,24.597730,314.064055,312.452928,51.409086,20.758512,-0.004910,0.032545
0,2019-01-02,124.029999,127.879997,123.000000,118.671188,4783200,CAT,2,-0.655001,336.668731,303.510025,49.792068,-33.729612,14.054906,314.492846,312.392662,51.409086,20.156977,-0.045157,0.041122
0,2019-01-02,42.279999,43.200001,42.209999,40.057236,23833500,CSCO,2,-0.159995,336.703986,304.162298,53.389344,31.217425,7.135987,315.392609,312.451774,51.409086,20.676912,-0.055490,0.024715


In [27]:

e_trade_gym.hmax = 2500





In [28]:
print(len(e_trade_gym.dates))

505


In [29]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=model,environment = e_trade_gym)

EPISODE|STEPS|TERMINAL_REASON|TOT_ASSETS|TERMINAL_REWARD_unsc|CASH_PCT  
   1| 238|CASH SHORTAGE  |$1135073  |-0.03511% |0.74%     
hit end!


In [30]:
df_account_value.shape

(238, 5)

In [31]:
df_account_value.head(50)

,cash,asset_value,total_assets,reward,date
0,1000000.000000,0.000000,1.000000e+06,0.000000,2020-01-24
1,976273.531939,23206.764662,9.994803e+05,0.000000,2020-01-27
2,951872.076738,49000.275615,1.000872e+06,-0.000260,2020-01-28
3,940005.671486,60941.721638,1.000947e+06,0.000291,2020-01-29
4,932008.029761,69548.279073,1.001556e+06,0.000237,2020-01-30
5,927623.939283,73840.253029,1.001464e+06,0.000311,2020-01-31
6,912247.381948,89513.900511,1.001761e+06,0.000244,2020-02-03
7,888919.480596,112653.872762,1.001573e+06,0.000252,2020-02-04
8,877143.455444,123829.504787,1.000973e+06,0.000197,2020-02-05
9,870134.705354,131920.061884,1.002055e+06,0.000108,2020-02-06


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [36]:
print("==============Get Backtest Results===========")
perf_stats_all = backtest_stats(account_value=df_account_value, value_col_name = 'total_assets')

==============Get Backtest Results===========
Annual return          0.141280
Cumulative returns     0.132932
Annual volatility      0.088964
Sharpe ratio           1.536520
Calmar ratio           2.888671
Stability              0.811245
Max drawdown          -0.048908
Omega ratio            1.326819
Sortino ratio          2.146971
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.131104
Daily value at risk   -0.010666
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [58]:
print("==============Compare to DJIA===========")
!pip uninstall finrl
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
from finrl.trade.backtest import backtest_plot, backtest_stats
backtest_plot(df_account_value, 
             baseline_ticker = 'crypto', 
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01', value_col_name = 'total_assets')

==============Compare to DJIA===========
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CRYPTO: No data found, symbol may be delisted


AttributeError: Can only use .dt accessor with datetimelike values

# Test against Crypto trading


In [35]:

cr_df = pd.read_csv('~/rldata/crypto-markets_top30.csv')
cr_df.head()

cr_fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

cr_processed = fe.preprocess_data(cr_df)


Successfully added technical indicators
Successfully added turbulence index
